In [1]:
#dumb installs
%load_ext autoreload
%autoreload 2

import os
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from keras.layers import MaxPooling2D
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from pandas import read_csv, merge
from PIL import Image
import matplotlib.pylab as plt
%matplotlib inline

2024-03-17 22:43:44.368711: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 22:43:44.402455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 22:43:44.402498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 22:43:44.403625: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 22:43:44.409373: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 22:43:44.409967: I tensorflow/core/platform/cpu_feature_guard.cc:1

2024-03-17 22:43:45.361329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),activation='relu',
input_shape=(69,69,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [4]:
data_path = "galaxyZoo/"
labels_file = 'gz2_hart16.csv' 

labels_df = read_csv(os.path.join(data_path, labels_file))
img_id_df = read_csv(os.path.join(data_path, "gz2_filename_mapping.csv"))

In [5]:
labels_df.rename(columns={'dr7objid': "specobjid"}, inplace=True)
relevant_columns = ['specobjid', 'gz2_class']  # Specify the columns you need

smaller_df = labels_df[relevant_columns].copy()  # Create a new DataFrame with only the relevant columns
# smaller_df = smaller_df.dropna()
smaller_df = smaller_df.sort_values(by = "specobjid")
print(smaller_df)

smaller_df["specobjid"] = smaller_df["specobjid"].astype(int)

img_id_df = img_id_df.dropna()
img_id_df = img_id_df.sort_values(by = "specobjid")
img_id_df = img_id_df.head(239695)



                 specobjid gz2_class
85359   587722981741363294        Sb
35562   587722981741363323      Sc?l
168093  587722981741428877  SBb3m(r)
212757  587722981741559888        Er
57863   587722981741625481      Sc1t
...                    ...       ...
102590  588848901537988759        Ei
180427  588848901538447575        Ei
185027  588848901538578615     SBb2m
231083  588848901539430566        Ec
81854   588848901539627254     SBc?m

[239695 rows x 2 columns]


In [6]:
merge_df = merge(smaller_df, img_id_df, on = "specobjid", how = "inner")
print(merge_df)

                 specobjid gz2_class    sample  asset_id
0       587722981741363294        Sb  original         3
1       587722981741363323      Sc?l  original         4
2       587722981741428877  SBb3m(r)     extra    245612
3       587722981741559888        Er  original         5
4       587722981741625481      Sc1t  original         6
...                    ...       ...       ...       ...
194947  588011219683180778      Sc?m  original    201978
194948  588011219683770432     Er(o)  original    201979
194949  588011219683836110        Er  original    201980
194950  588011219683967294        Er  original    201981
194951  588011219684229338      Sc?t  original    201982

[194952 rows x 4 columns]


In [7]:
img_name = os.listdir("galaxyZoo/images/")

asset_id = []

for name in img_name:
    name = name.removesuffix(".jpg")
    name = int(name)
    asset_id.append(name)
#14060

mask = merge_df["asset_id"].isin(asset_id)
final_df = merge_df[mask]
print(final_df.shape)
final_df = final_df.sort_values(by = "asset_id")
print(final_df)
#final_df.to_csv("please.csv")

(13965, 4)
                 specobjid gz2_class    sample  asset_id
8       587722981741756545        Sb  original        11
101     587722981748113567   Sc?t(r)  original       100
102     587722981748179130      Sc+t  original       101
103     587722981748375756     SBc?t  original       102
104     587722981748375801        Ei  original       103
...                    ...       ...       ...       ...
113121  587738947199893702        Sc  original    115401
113122  587738947200024706       Ser  original    115402
113123  587738947200024811        Ei  original    115403
113124  587738947200155873        Ei  original    115404
113125  587738947200352424        Er  original    115405

[13965 rows x 4 columns]


In [8]:
filenames_in_mapping = final_df['asset_id'].tolist()
filenames_in_mapping = [str(file) + ".jpg" for file in filenames_in_mapping]

count = 0
for filename in os.listdir("galaxyZoo/images/"):
    if filename not in filenames_in_mapping:
        file_path = os.path.join("galaxyZoo/images/", filename)
        os.remove(file_path)
        count += 1

print(count)
        

0


https://github.com/jameslawlor/kaggle_galaxy_zoo/blob/master/galaxy_zoo_keras.ipynb


In [9]:
spec_list = []
class_df = final_df["gz2_class"]
for n in class_df:
    spec_list.append(n)
spec_list = [s[:2] for s in spec_list]
#batch because laptop doesn't have enough memory
spec_list = spec_list[:4805]
print(spec_list)

label_encoder = LabelEncoder()
specs = label_encoder.fit_transform(spec_list)



In [10]:
n = []
for letters in spec_list:
    if letters not in n:
        n.append(letters)
    else:
        continue
print(n)

['Sb', 'Sc', 'SB', 'Ei', 'Er', 'Se', 'Ec', 'Sd', 'A', 'Sa']


In [11]:
x = []
#np.empty((424, 424, 3))
n = 0
while n < 4805: 
    for filename in os.listdir("galaxyZoo/images/"):
        image = Image.open(f"galaxyZoo/images/{filename}")
        img_array = np.array(image) 
        x.append(img_array)
        n +=1
        if n >= 4805:
            break
x = np.array(x)
print(x.shape)
#img_df = DataFrame({"Image": x})

(4805, 424, 424, 3)


In [12]:
# y = np.array(final_df["gz2_class"])
# print(img_dataset.dtype)
# images_train, images_test, labels_train, labels_test = train_test_split(img_dataset, y, test_size=0.2, random_state=42)

In [13]:
specs = to_categorical(specs)
x_train, x_test, y_train, y_test = train_test_split(x, specs, test_size=0.2, random_state=42)
print(x_train.shape)
# np.array(x_train)
# np.array(x_test)
# np.array(y_train)
# np.array(y_test)


(3844, 424, 424, 3)


In [14]:
# model = Sequential([
#   Rescaling(1./255),
#   Conv2D(32, 3, activation='relu'),
#   MaxPooling2D(),
#   Conv2D(32, 3, activation='relu'),
#   MaxPooling2D(),
#   Conv2D(32, 3, activation='relu'),
#   MaxPooling2D(),
#   Flatten(),
#   Dense(128, activation='relu'),
#   Dense(1)
# ])

In [15]:
model = Sequential([
    Rescaling(1./255),
    Flatten(input_shape=(424, 424, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='relu')
])

In [16]:
# t_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)
model.fit(
    x_train,
    y_train,
    epochs = 15, 
    validation_data = (x_test, y_test),
    batch_size= 100,
)


Epoch 1/15


2024-03-17 22:43:59.017319: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2073176832 exceeds 10% of free system memory.


39/39 [==============================] - 18s 441ms/step - loss: 7.4913 - accuracy: 0.1904 - val_loss: 7.0452 - val_accuracy: 0.1727
Epoch 2/15
39/39 [==============================] - 17s 435ms/step - loss: 7.3413 - accuracy: 0.1842 - val_loss: 7.0463 - val_accuracy: 0.1988
Epoch 3/15
39/39 [==============================] - 17s 438ms/step - loss: 7.3402 - accuracy: 0.1990 - val_loss: 7.0447 - val_accuracy: 0.1967
Epoch 4/15
39/39 [==============================] - 17s 433ms/step - loss: 7.3367 - accuracy: 0.1977 - val_loss: 7.0431 - val_accuracy: 0.1863
Epoch 5/15
39/39 [==============================] - 17s 433ms/step - loss: 7.3335 - accuracy: 0.2071 - val_loss: 7.0476 - val_accuracy: 0.1956
Epoch 6/15
39/39 [==============================] - 17s 435ms/step - loss: 7.3264 - accuracy: 0.2357 - val_loss: 7.0628 - val_accuracy: 0.1842
Epoch 7/15
39/39 [==============================] - 17s 438ms/step - loss: 7.4374 - accuracy: 0.2326 - val_loss: 7.5173 - val_accuracy: 0.1904
Epoch 8/15

In [19]:
img = Image.open("galaxyZoo/images/115405.jpg")
img = np.array(img)
img = np.reshape(img, (1, 424, 424, 3))
predict = model.predict(img)
print(predict)

1/1 [==============================] - 0s 54ms/step
[[  0.       96.32027   0.      385.3395  396.27444   0.        0.
  441.47107   0.        0.     ]]
